In [5]:
from typing import Type

import requestcompletion as rc
from pydantic import BaseModel, Field


In [ ]:
class ResponseModel(BaseModel):
    quality: float = Field(description="The quality of the statement on a scale of 0 to 1")
    completeness: float = Field(description="The completeness of the statement on a scale of 0 to 1")
    understanding: str = Field (description="Your understanding of the statement")
    suggestions: str = Field(description="Suggestions for improvement")
    overall_score: float = Field(description="The overall score of the statement on a scale of 0 to 1")

    

class StatementAnalyzer(rc.nodes.library.StructuredLLM[ResponseModel]):
    @classmethod
    def pretty_name(cls) -> str:
        return "Statement Analyzer"
    
    @classmethod
    def output_model(cls):
        return ResponseModel
    
    system_prompt = """You are a master of the english langauge desigend to analyze the success of a statement given to you. You may be provided with context or you may be provided none. Do what you can with the information provided."""
    
    model = lambda x: rc.llm.models.OpenAILLM("gpt-4o")

    def __init__(self, message_history: rc.llm.MessageHistory):
        message_history.insert(0, rc.llm.SystemMessage(self.system_prompt))
        super().__init__(
            message_history=message_history,
            model= self.model(),
        )
        
    

In [15]:
prompt = "Analyze the following statement \n 'The quick brown fox jumps over the lazy dog'. \n It should be consice and clear with no ambiguity."

sa = StatementAnalyzer(message_history=rc.llm.MessageHistory([rc.llm.UserMessage(prompt)]))

result = rc.run.run(sa)

obj = result.answer

In [16]:

print(isinstance(obj, ResponseModel))
print(obj.model_dump_json(indent=2))

True
{
  "quality": 0.9,
  "completeness": 1.0,
  "understanding": "The statement is a well-known pangram in the English language, meaning it contains every letter of the alphabet at least once. It is concise and clear, with no ambiguity.",
  "suggestions": "The statement is already concise and clear. No improvements are necessary.",
  "overall_score": 0.95
}
